In [ ]:
# Import dependencies

import time
import os.path

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np

%matplotlib inline

import ROOT

import root_pandas as rpd
from root_pandas import read_root

import math
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

import matplotlib.ticker as mticker

formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3,2))

In [ ]:
def getJetData(inputFiles):
    
    # These exist in the tree named tnj
    scalarColumns = []
    scalarColumns.append('Q')
    scalarColumns.append('x')
    scalarColumns.append('y')
    scalarColumns.append('lpt')
    scalarColumns.append('e')
    scalarColumns.append('m')
    scalarColumns.append('pt')
    scalarColumns.append('phi')
    scalarColumns.append('eta')
    scalarColumns.append('mult')
    scalarColumns.append('sd_mult')
    scalarColumns.append('zg')
    scalarColumns.append('delphi')
    scalarColumns.append('xsec')
    scalarColumns.append('ntrials')

    arrayColumns = []

    columns = scalarColumns + arrayColumns

    dfs = []
    for inputFile in inputFiles:
        start = time.time()
        filename = os.path.basename(inputFile)
        dataframe = rpd.read_root(inputFile, 'tnj', columns = columns, flatten = arrayColumns) 
        end = time.time()
        print 'Processed {0} in {1} seconds'.format(filename, end - start)
        dfs.append(dataframe)

    return pd.concat(dfs).drop_duplicates().reset_index(drop=True)

In [ ]:
def getEventData(inputFiles):
    
    # These exist in the tree named tne
    scalarColumns = []
    scalarColumns.append('Q')
    scalarColumns.append('W')
    scalarColumns.append('x')
    scalarColumns.append('y')
    scalarColumns.append('xsec')
    scalarColumns.append('ntrials')

    arrayColumns = []

    columns = scalarColumns + arrayColumns

    dfs = []
    for inputFile in inputFiles:
        start = time.time()
        filename = os.path.basename(inputFile)
        dataframe = rpd.read_root(inputFile, 'tne', columns = columns, flatten = arrayColumns) 
        end = time.time()
        print 'Processed {0} in {1} seconds'.format(filename, end - start)
        dfs.append(dataframe)

    return pd.concat(dfs).drop_duplicates().reset_index(drop=True)

In [ ]:
# Get the data from a .root file, which was made in the same code that was used to generate the events.

filename   = ['/Users/jose/EIC/data/eic_dis.root']

data       = getJetData(filename)
event_data = getEventData(filename)

In [ ]:
# Need to retreive the number of trials before we cut. This is needed for normalizing to the cross section.

ntrials = data['ntrials'][data.shape[0]-1]

In [ ]:
# The variable Q2 is added to the dataframe, and we apply an inelasticity cut to both sets of data.

data.eval('Q2 = Q*Q', inplace = True)
event_data.eval('Q2 = Q*Q', inplace = True)

data       = data.query('0.01 < y < 0.95')
event_data = event_data.query('0.01 < y < 0.95')

In [ ]:
# These control our labels on the graph. Sometimes we change things and this should minimize annoyances.

# True makes the center mass of energy of the plot be 141 GeV with elabel meaning e beam energy...

energy = True

slabel = ''
elabel = ''
plabel = ''
algolabel = ''

if energy == True:
    slabel = str(141)
    elabel = str(20)
    plabel = str(250)
    algolabel = str('Cambridge/Aachen') #r'$\mathrm{anti}-k_{\mathrm{T}}$'
    
else:
    slabel = str(90)
    elabel = str(20)
    plabel = str(100)
    algolabel = str('Cambridge/Aachen')

In [ ]:
# The following def normalizes our distribution to the cross section (normed=False) with other functionality.

def plotData(array, bins, arange = None, color = 'red', fillstyle = 'full', markersize = '2', normed = None, label = ''):
    
    counts, edges = np.histogram(array, bins = bins, range = arange)
    binWidth = edges[1] - edges[0]
    
    # Consider a luminosity of 10/fb, also notice the 1e12 factor results in a cross section in fb.
    yerr = np.sqrt( 10 * (counts * np.mean(data['xsec']) * 1e12) / ntrials )
    
    if normed == True:
    
        plt.errorbar(edges[:-1] + binWidth / 2, 10 * (counts * np.mean(data['xsec']) * 1e12) / ntrials,
                     xerr = binWidth / 2, yerr = yerr, color = color, markersize = markersize, 
                     fillstyle = fillstyle, fmt = 'o', label = label)
    
    # Here a factor of 1e9 is used in order to compare to other studies that use units of pb instead of fb.
    else:
        plt.errorbar(edges[:-1] + binWidth / 2, (counts * np.mean(data['xsec']) * 1e9) / (binWidth*ntrials), 
                     xerr = binWidth / 2, 
                     yerr = np.sqrt(counts) * np.mean(data['xsec']) * 1e9 / (binWidth*ntrials), 
                     color = color, markersize = markersize, fillstyle = fillstyle, fmt = 'o', label = label)

In [ ]:
# We are ready to get an idea of the distributions of different variables. We start with jet pT.

fig,ax = plt.subplots(figsize=(18,8))

plt.subplot(1,2,1)
plotData(data.query('abs(eta) < 2')['pt'], bins = 25, arange = (5,55))

plt.plot([],[],'',label=r'$\mathrm{DIS \: configuration \: e-p}$', color='white')
plt.plot([],[],'',label=r'$\mathrm{E}_{beam}^{e^{-}} \: = ' + elabel + ' \: \mathrm{GeV}, \: \mathrm{E}_{beam}^{p^{+}} \: = ' + plabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \: > \: 5 \:\mathrm{GeV}/c$', color='white')
plt.plot([],[],'',label=r'$|\eta_{\mathrm{jet}}| \: <  \: 2$', color='white')

plt.xlabel(r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \:\:(\mathrm{GeV}/c)$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(5,35)
plt.ylabel(r'$\mathrm{d} \sigma \:/\: \mathrm{d} \mathrm{p}_\mathrm{T}^\mathrm{jet} \: (\mathrm{pb}/\mathrm{GeV})$', fontsize = 28)
plt.yscale("log")
plt.yticks(fontsize = 17)
plt.ylim(1,10000)
plt.minorticks_on()
plt.legend(loc='upper right', frameon=False, prop={'size':15})

plt.subplot(1,2,2)
plotData(data.query('abs(eta) < 2')['pt'], bins = 25, arange = (5,55), normed = True)

plt.plot([],[],'',label=r'$\mathrm{DIS \: configuration \: e-p}$', color='white')
plt.plot([],[],'',label=r'$\mathrm{E}_{beam}^{e^{-}} \: = ' + elabel + ' \: \mathrm{GeV}, \: \mathrm{E}_{beam}^{p^{+}} \: = ' + plabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \: > \: 5 \:\mathrm{GeV}/c$', color='white')
plt.plot([],[],'',label=r'$|\eta_{\mathrm{jet}}| \: <  \: 2$', color='white')

plt.xlabel(r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \:\:(\mathrm{GeV}/c)$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(5,35)
plt.ylabel(r'$\mathrm{d} \sigma \: \times \: \mathrm{10} \: \mathrm{fb^{-1}}$', fontsize = 28)
plt.yscale("log")
plt.yticks(fontsize =17)
plt.ylim(20000,1000000000)
plt.minorticks_on()
plt.legend(loc='upper right', frameon=False, prop={'size':15})

In [ ]:
# Now let us see the jets pT after applying a cut on the scattered electrons pT.

fig,ax = plt.subplots(figsize=(8,7))

plotData(data.query('lpt > 10')['pt'], bins = 25, arange = (5,30), normed = True)

plt.plot([],[],'',label=r'$\mathrm{DIS \: configuration \: e-p}$', color='white')
plt.plot([],[],'',label=r'$\mathrm{E}_{beam}^{e^{-}} \: = ' + elabel + ' \: \mathrm{GeV}, \: \mathrm{E}_{beam}^{p^{+}} \: = ' + plabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \: > \: 5 \:\mathrm{GeV}/c$', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{scat. e} \: > \: 10 \:\mathrm{GeV}/c$', color='white')

plt.xlabel(r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \:\:(\mathrm{GeV}/c)$', fontsize = 23)
plt.xticks(fontsize = 17)
# plt.xlim(9,30)
plt.ylabel(r'$\mathrm{d} \sigma \: \times \: \mathrm{10} \: \mathrm{fb^{-1}}$', fontsize = 26)
plt.yticks(fontsize = 17)
plt.minorticks_on()
plt.legend(loc='upper right', frameon=False, prop={'size':14})
ax.yaxis.set_major_formatter(formatter)
# plt.savefig('ep_sim_jet_pT_lptCut.pdf')

In [ ]:
# Smear the jet pT according to JER as seen from sPHENIX.

i = 0

smear_pT = np.zeros(data.shape[0])

def jetEnergyResol(pT):
    
    return (0.2 * pT)

for index, row in data.iterrows():
    
    sigma_det   = np.abs(jetEnergyResol(row['pt']))
    smear_pT[i] = np.random.normal(0, sigma_det, 1)
    
    i = i + 1

data['smear_pt'] = smear_pT
data.eval('smeared_pt = pt + smear_pt',inplace = True)

fig,ax = plt.subplots(figsize=(8,7))

plt.hist(data['pt'],bins=50,range=(0,40),color='red',label='Unsmeared',alpha=0.5);
plt.hist(data['smeared_pt'],bins=50,range=(0,40),color='blue',label='Smeared',alpha=0.5);
plt.xlabel(r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \:\:(\mathrm{GeV}/c)$',fontsize = 26)
plt.xticks(fontsize = 17);
plt.ylabel('Counts', fontsize = 23)
plt.yticks(fontsize = 17);
plt.legend(loc='upper right', frameon=False, prop={'size':16})

In [ ]:
# Let us continue by looking at the jet psuedorapidity distributions.

fig,ax = plt.subplots(figsize=(35,15))

plt.subplot(1,2,1)
plotData(data.query('pt > 10')['eta'], bins = 15, arange = (-1,4))

plt.plot([],[],'',label=r'$\mathrm{DIS \: configuration \: e-p}$', color='white')
plt.plot([],[],'',label=r'$\mathrm{E}_{beam}^{e^{-}} \: = ' + elabel + ' \: \mathrm{GeV}, \: \mathrm{E}_{beam}^{p^{+}} \: = ' + plabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \: > \: 10 \:\mathrm{GeV}/c$', color='white')

plt.xlabel(r'$\eta_{\mathrm{jet}}$', fontsize = 36)
plt.xticks(fontsize = 23)
plt.xlim(-1, 4)
plt.ylabel(r'$\mathrm{d} \sigma \: / \: \mathrm{d} \eta_\mathrm{jet} \: (\mathrm{fb})$', fontsize = 34)
plt.yticks(fontsize = 23)
# plt.axvline(x=0.5)
plt.minorticks_on()
plt.legend(loc='upper right', frameon=False, prop={'size':25})
ax.yaxis.set_major_formatter(formatter)

plt.subplot(1,2,2)
plotData(data.query('lpt > 10')['eta'], bins = 25, arange = (-1.5,6), normed = True)

plt.plot([],[],'',label=r'$\mathrm{DIS \: configuration \: e-p}$', color='white')
plt.plot([],[],'',label=r'$\mathrm{E}_{beam}^{e^{-}} \: = ' + elabel + ' \: \mathrm{GeV}, \: \mathrm{E}_{beam}^{p^{+}} \: = ' + plabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \: > \: 5 \:\mathrm{GeV}/c$', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{scat. e} \: > \: 10 \:\mathrm{GeV}/c$', color='white')

plt.xlabel(r'$\eta_{\mathrm{jet}}$', fontsize = 36)
plt.xticks(fontsize = 23)
plt.xlim(-1.5, 4)
plt.ylabel(r'$\mathrm{d} \sigma \: \times \: \mathrm{10} \: \mathrm{fb^{-1}}$', fontsize = 34)
plt.yticks(fontsize = 23)
plt.minorticks_on()
plt.legend(loc='upper left', bbox_to_anchor=(0.8, 1), prop={'size':25})
# plt.savefig('ep_sim_jet_eta_lptCut.pdf')

In [ ]:
# Smear the jet eta (psuedorapidity) distribution.

j = 0

smear_eta = np.zeros(data.shape[0])

def jetEtaResol(eta):
    
    return (0.2 / np.tanh(eta))

for index, row in data.iterrows():

    #sigma_det    = np.abs( jetEtaResol(row['eta']) )
    #smear_eta[j] = np.random.normal(0, sigma_det, 1)
    
    smear_eta[j] = row['eta'] * 1.462
    j = j + 1

data['smear_eta'] = smear_eta
data.eval('smeared_eta = smear_eta',inplace = True)

fig,ax = plt.subplots(figsize=(8,7))

plt.hist(data['eta'],bins=50,color='red',label='Unsmeared',alpha=0.5);
plt.hist(data['smeared_eta'],bins=50,range=(-2,3.5),color='blue',label='Smeared',alpha=0.5);
plt.xlabel(r'$\eta_\mathrm{jet}$', fontsize = 26)
plt.xticks(fontsize = 17);
plt.ylabel('Counts', fontsize = 23)
plt.yticks(fontsize = 17);
plt.legend(loc='upper right', frameon=False, prop={'size':16})

In [ ]:
# In order to smear jet eta, we need to quantify the std of the gaussian (above we use 0.462).

j = 0

test_smear_eta = np.zeros(data.shape[0])

for index, row in data.iterrows():

    sigma_det         = np.abs( jetEtaResol(row['eta']) )
    test_smear_eta[j] = np.random.normal(0, sigma_det, 1)
    
    j = j + 1

data['test_smear_eta'] = test_smear_eta
data.eval('test_smeared_eta = test_smear_eta + eta',inplace = True)
    
counts, edges = np.histogram(data['eta'] - data['test_smeared_eta'], bins = 50, range = (-0.8,0.8))
binWidth = edges[1] - edges[0]    

x = edges[:-1] + binWidth / 2
y = counts

n = len(y)                          
mean = sum(x)/n                   
sigma = math.sqrt(sum((x-mean)**2)/n)

def gaus(x,a,x0,sigma,offset):
    return a*exp(-(x-x0)**2/(2*sigma**2)) + offset

fig,ax = plt.subplots(figsize=(8,7))

popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma,0.0])
plt.xlabel(r'$\eta \: - \: \eta_{smeared}$',fontsize=26)
plt.xticks(fontsize = 17);
plt.ylabel('Counts', fontsize=23)
plt.yticks(fontsize = 17);
plt.errorbar(x,y,xerr =(edges[1]-edges[0])/2,yerr=np.sqrt(y),label='data')
plt.plot(x,gaus(x,*popt),'ro:',label='fit')

# Determined chi^2 value

plt.plot([],[],color='white',label = 'reduced chi-squared =' + str(round(sum(( (gaus(x,*popt) - y) / np.sqrt(y) )**2) / (n - 5),2)) + '')
print 'Sigma:', sigma
plt.legend(loc='upper right')

plt.show()

In [ ]:
# Move on to smearing jet phi distributions (*in the works*).

k = 0

smear_phi = np.zeros(data.shape[0])

def jetPhiResol(phi):
    
    return (0.2 * -1 * np.tanh(phi))

for index, row in data.iterrows():

    sigma_det    = np.abs( jetPhiResol(row['phi']) )
    smear_phi[k] = np.random.normal(0, sigma_det, 1)
    
    k = k + 1

data['smear_phi'] = smear_phi
data.eval('smeared_phi = phi + smear_phi',inplace = True)

fig,ax = plt.subplots(figsize=(8,7))

plt.hist(data['phi'],bins=50,color='red',label='Unsmeared',alpha=0.5);
plt.hist(data['smeared_phi'],bins=50,range=(0,6.28),color='blue',label='Smeared',alpha=0.5);
plt.xlabel(r'$\phi_\mathrm{jet}$', fontsize = 26)
plt.xticks(fontsize = 17);
plt.ylabel('Counts', fontsize = 23)
plt.yticks(fontsize = 17);
plt.ylim(0,4500)
plt.legend(loc='upper right', frameon=False, prop={'size':16})

In [ ]:
# Now we develop 2D plots utilizing some of the variables plotted above and also consider others.

xmin = np.amin(event_data['x'])
xmax = np.amax(event_data['x'])
ymin = np.amin(event_data['Q2'])
ymax = np.amax(event_data['Q2'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(event_data['x'], event_data['Q2'],xscale='log',yscale='log', norm=matplotlib.colors.LogNorm(1,10000), gridsize=40, cmap='inferno')
ax.axis([xmin, xmax, ymin, ymax], fontsize=17)
ax.set_xlabel(r'$\mathrm{x}$', fontsize = 20)
ax.set_ylabel('$\mathrm{Q}^2 \:\: (\mathrm{GeV}^2)$', fontsize=20)
cb = fig.colorbar(hb, ax=ax)
cb.ax.tick_params(labelsize=15)
cb.set_label('Counts', fontsize=20)
plt.show()

In [ ]:
xmin = np.amin(data.query('5 < pt < 10')['x'])
xmax = np.amax(data.query('5 < pt < 10')['x'])
ymin = np.amin(data.query('5 < pt < 10')['Q2'])
ymax = np.amax(data.query('5 < pt < 10')['Q2'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(data.query('5 < pt < 10')['x'], data.query('5 < pt < 10')['Q2'],xscale='log',yscale='log', norm=matplotlib.colors.LogNorm(1,1000), gridsize=40, label='hi', cmap='inferno')
ax.axis([xmin, xmax, ymin, ymax], fontsize=17)
ax.set_xlabel(r'$\mathrm{x}$', fontsize = 20)
ax.set_ylabel('$\mathrm{Q}^2 \:\: (\mathrm{GeV}^2)$', fontsize=20)
cb = fig.colorbar(hb, ax=ax)
cb.ax.tick_params(labelsize=15)
cb.set_label('Counts', fontsize=20)
plt.show()

In [ ]:
xmin = np.amin(data.query('10 < pt < 15')['x'])
xmax = np.amax(data.query('10 < pt < 15')['x'])
ymin = np.amin(data.query('10 < pt < 15')['Q2'])
ymax = np.amax(data.query('10 < pt < 15')['Q2'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(data.query('10 < pt < 15')['x'], data.query('10 < pt < 15')['Q2'],xscale='log',yscale='log', norm=matplotlib.colors.LogNorm(1,1000), gridsize=40, label='hi', cmap='inferno')
ax.axis([xmin, xmax, ymin, ymax], fontsize=17)
ax.set_xlabel(r'$\mathrm{x}$', fontsize = 20)
ax.set_ylabel('$\mathrm{Q}^2 \:\: (\mathrm{GeV}^2)$', fontsize=20)
cb = fig.colorbar(hb, ax=ax)
cb.ax.tick_params(labelsize=15)
cb.set_label('Counts', fontsize=20)
plt.show()

In [ ]:
xmin = np.amin(data.query('15 < pt < 20')['x'])
xmax = np.amax(data.query('15 < pt < 20')['x'])
ymin = np.amin(data.query('15 < pt < 20')['Q2'])
ymax = np.amax(data.query('15 < pt < 20')['Q2'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(data.query('15 < pt < 20')['x'], data.query('15 < pt < 20')['Q2'],xscale='log',yscale='log', norm=matplotlib.colors.LogNorm(1,1000), gridsize=40, label='hi', cmap='inferno')
ax.axis([xmin, xmax, ymin, ymax], fontsize=17)
ax.set_xlabel(r'$\mathrm{x}$', fontsize = 20)
ax.set_ylabel('$\mathrm{Q}^2 \:\: (\mathrm{GeV}^2)$', fontsize=20)
cb = fig.colorbar(hb, ax=ax)
cb.ax.tick_params(labelsize=15)
cb.set_label('Counts', fontsize=20)
plt.show()

In [ ]:
xmin = np.amin(data.query('20 < pt < 30')['x'])
xmax = np.amax(data.query('20 < pt < 30')['x'])
ymin = np.amin(data.query('20 < pt < 30')['Q2'])
ymax = np.amax(data.query('20 < pt < 30')['Q2'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(data.query('20 < pt < 30')['x'], data.query('20 < pt < 30')['Q2'],xscale='log',yscale='log', norm=matplotlib.colors.LogNorm(1,1000), gridsize=40, label='hi', cmap='inferno')
ax.axis([xmin, xmax, ymin, ymax], fontsize=17)
ax.set_xlabel(r'$\mathrm{x}$', fontsize = 20)
ax.set_ylabel('$\mathrm{Q}^2 \:\: (\mathrm{GeV}^2)$', fontsize=20)
cb = fig.colorbar(hb, ax=ax)
cb.ax.tick_params(labelsize=15)
cb.set_label('Counts', fontsize=20)
plt.show()

In [ ]:
xmin = np.amin(data['eta'])
xmax = np.amax(data['eta'])
ymin = np.amin(data['e'])
ymax = np.amax(data['e'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(data['eta'], data['e'], yscale='log', gridsize=40, norm=matplotlib.colors.LogNorm(1,1000), cmap='inferno', mincnt=1)
ax.axis([xmin, xmax, ymin, ymax], fontsize=17)
ax.set_ylabel(r'$\mathrm{E}_\mathrm{jet} \:\:(\mathrm{GeV})$', fontsize = 20)
plt.xticks(fontsize = 15)
ax.set_xlabel('$\eta_\mathrm{jet}$', fontsize=20)
plt.yticks(fontsize = 15)
cb = fig.colorbar(hb, ax=ax)
cb.ax.tick_params(labelsize=15)
cb.set_label('Counts', fontsize=20)
# plt.savefig('ep_sim_jet_eta_E.pdf')

In [ ]:
xmin = np.amin(data['x'])
xmax = np.amax(data['x'])
ymin = np.amin(data['pt'])
ymax = np.amax(data['pt'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(data['x'], data['pt'], xscale='log', gridsize=25, norm=matplotlib.colors.LogNorm(1,8000), cmap='inferno', mincnt=1)
ax.axis([xmin, xmax, ymin, ymax], fontsize=17)
ax.set_ylabel(r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \:\:(\mathrm{GeV}/c)$', fontsize = 20)
plt.xticks(fontsize = 15)
ax.set_xlabel('$\mathrm{x}$', fontsize=20)
plt.yticks(fontsize = 15)
cb = fig.colorbar(hb, ax=ax)
cb.ax.tick_params(labelsize=15)
cb.set_label('Counts', fontsize=20)
plt.show()

In [ ]:
xmin = np.amin(data['x'])
xmax = np.amax(data['x'])
ymin = np.amin(data['e'])
ymax = np.amax(data['e'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(data['x'], data['e'], xscale='log', gridsize=25, norm=matplotlib.colors.LogNorm(1,10000), cmap='inferno', mincnt=1)
ax.axis([xmin, xmax, ymin, ymax], fontsize=17)
ax.set_ylabel(r'$\mathrm{E}_\mathrm{jet} \:\:(\mathrm{GeV})$', fontsize = 20)
plt.xticks(fontsize = 15)
ax.set_xlabel('$\mathrm{x}$', fontsize=20)
plt.yticks(fontsize = 15)
cb = fig.colorbar(hb, ax=ax)
cb.ax.tick_params(labelsize=15)
cb.set_label('Counts', fontsize=20)
plt.show()

In [ ]:
xmin = np.amin(data['eta'])
xmax = np.amax(data['eta'])
ymin = np.amin(data['x'])
ymax = np.amax(data['x'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(data['eta'], data['x'], yscale='log', norm=matplotlib.colors.LogNorm(1,3000), gridsize=25, cmap='inferno', mincnt=1)
ax.axis([xmin, xmax, -2, ymax], fontsize=17)
ax.set_ylabel(r'$\mathrm{x}$', fontsize = 20)
plt.yticks(fontsize = 15)
ax.set_xlabel('$\eta_\mathrm{jet}$', fontsize=20)
plt.xticks(fontsize = 15)
cb = fig.colorbar(hb, ax=ax)
cb.ax.tick_params(labelsize=15)
cb.set_label('Counts', fontsize=20)
plt.show()

In [ ]:
xmin = np.amin(data['e'])
xmax = np.amax(data['e'])
ymin = np.amin(data['pt'])
ymax = np.amax(data['pt'])

fig, axs = plt.subplots(figsize=(7, 7))
fig.subplots_adjust(hspace=0.5, left=0.07, right=0.93)
ax = axs

hb = ax.hexbin(data['e'], data['pt'], gridsize=20, norm=matplotlib.colors.LogNorm(1,10000), cmap='inferno',mincnt=1)
ax.axis([xmin, xmax, ymin, ymax], fontsize=17)
ax.set_ylabel(r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \:\:(\mathrm{GeV}/c)$', fontsize = 20)
plt.xticks(fontsize = 15)
ax.set_xlabel(r'$\mathrm{E}_\mathrm{jet} \:\:(\mathrm{GeV})$', fontsize=20)
plt.yticks(fontsize = 15)
cb = fig.colorbar(hb, ax=ax)
cb.ax.tick_params(labelsize=15)
cb.set_label('Counts', fontsize=20)
plt.show()

In [ ]:
# We go deeper by looking at the jet substructure. To do so, we look at zG distribution.

fig = plt.figure(figsize=(9,7))

plotData(data['zg'], 100)
plt.plot([],[],'',label=r'$\mathrm{DIS \: configuration \: e-p}$', color='white')
plt.plot([],[],'',label=r'$\mathrm{E}_{beam}^{e^{-}} \: = ' + elabel + ' \: \mathrm{GeV}, \: \mathrm{E}_{beam}^{p^{+}} \: = ' + plabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{SoftDrop} \: \: z_{\mathrm{min}} \: = \: 0.1, \: \beta \: = \: 0.0$', color='white')
plt.xlabel(r'$\mathrm{z}_{G}$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(0,0.6)
plt.ylabel(r'$\mathrm{d} \sigma \: / \: \mathrm{d} \mathrm{z}_{G} \: (\mathrm{fb.})$', fontsize = 28)
plt.yticks(fontsize =17)
plt.yscale('log')

plt.legend(loc='upper right', frameon=False, prop={'size':15})

In [ ]:
# zG distribution is split into eta ranges.

fig = plt.figure(figsize=(9,8))

plotData(data.query('eta < -1.0')['zg'], bins = 20, arange=(0,0.6), label = r'$ -1.0 > \eta$', color = 'purple', normed = True)
plotData(data.query('-1. < eta < 0.0')['zg'], bins = 20, arange=(0,0.6), label = r'$ -1.0 \: < \: \eta \: < \: 0.0$', color = 'green', normed = True)
plotData(data.query('0.0 < eta < 1.0')['zg'], bins = 20, arange=(0,0.6), label = r'$0.0 \: < \eta \: < \: 1.0$', fillstyle = 'none',color = 'blue', normed = True)
plotData(data.query('1.0 < eta < 2.0')['zg'], bins = 20, arange=(0,0.6), label = r'$1.0 \: < \eta \: < \: 2.0$', color = 'orange', normed = True)
plotData(data.query('2.0 < eta')['zg'], bins = 20, arange=(0,0.6), label = r'$\eta \: > \: 2.0$', color = 'red', normed = True)
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \: > \: 5 \:\mathrm{GeV}/c$', color='white')
plt.plot([],[],'',label=r'$\mathrm{SoftDrop} \: \: z_{\mathrm{min}} \: = \: 0.1, \: \beta \: = \: 0.0$', color='white')

plt.xlabel(r'$\mathrm{z}_{G}$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(0,0.6)
plt.ylabel(r'$\mathrm{d} \sigma  \: \times \: \mathrm{10} \: \mathrm{fb^{-1}}$', fontsize = 28)
plt.yticks(fontsize =17)
plt.yscale('log')

plt.legend(loc='upper left', bbox_to_anchor=(0.82, 1), prop={'size':15})
ax.yaxis.set_major_formatter(formatter)
# plt.savefig('ep_sim_zG_etaRanges.pdf')

In [ ]:
# Lepton-jet correlation from the event generations will be compared to theory.

fig,ax  = plt.subplots(figsize=(9,7))

plotData(data.query('lpt > 10')['delphi'], bins = 22, arange=(0,0.5))
plt.plot([],[],'',label=r'$\mathrm{DIS \: configuration \: e-p}$', color='white')
plt.plot([],[],'',label=r'$\mathrm{E}_{beam}^{e^{-}} \: = ' + elabel + '\: \mathrm{GeV}, \: \mathrm{E}_{beam}^{p^{+}} \: = ' + plabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \: > \: 5 \:\mathrm{GeV}/c$', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{scat. e} \: > \: 10 \:\mathrm{GeV}/c$', color='white')

plt.xlabel(r'$\Delta \phi \:\:(\mathrm{rad.})$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(0,0.5)
plt.ylabel(r'$\mathrm{d} \sigma \:/\: \mathrm{d} \Delta \phi \: (\mathrm{fb}/\mathrm{rad})$', fontsize = 28)
plt.yticks(fontsize =17)
plt.minorticks_on()
plt.legend(loc='upper right', frameon=False, prop={'size':15})
ax.yaxis.set_major_formatter(formatter)
# plt.savefig('ep_sim_jetElec_delphi_lptCut.pdf')

In [ ]:
def getRelStatError(array, bins, arange = None):

    counts, edges = np.histogram(array, bins = bins, range = arange)
    binWidth = edges[1] - edges[0]
    
    return np.sqrt( 10 * (counts * np.mean(data['xsec']) * 1e12) / ntrials ) / ( 10 * (counts * np.mean(data['xsec']) * 1e12) / ntrials ), edges

In [ ]:
stat_error, edges = getRelStatError(data.query('lpt > 10')['delphi'], bins = 22, arange=(0,0.5))
sys_error  = np.array([0.02]*(stat_error.shape[0]))

tot_error = np.sqrt( pow(stat_error,2) + pow(sys_error,2))

In [ ]:
theory_data = np.loadtxt("/Users/jose/EIC/lepton-jet-sivers/fig4.txt", skiprows=1)
delphi = theory_data[:,0][0:22]
qL_0   = theory_data[:,1][0:22]
qL_02  = theory_data[:,2][0:22]
qL_08  = theory_data[:,3][0:22]

one_arr = np.array([1] * delphi.shape[0])

fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(111)

plt.plot(delphi, qL_02 / qL_0,  'ro')
plt.errorbar(delphi, one_arr, yerr=tot_error, color='blue', markersize=1, capsize=3, fmt='o')
             
plt.ylim(0.92,1.06)
plt.yticks(fontsize = 17)
plt.xticks(fontsize = 17)
plt.axhline(y=1,color='grey',linestyle='--')

In [ ]:
# A def that will bin data (*a way using a for loop should be explored at some point*).

def bins(inputDataframe, variable, array, i):
    
        i = i - 1
        
        if i == 1:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i]) +     '')
            
            return df1
        if i == 2:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2
        if i == 3:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3
        if i == 4:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4
        if i == 5:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5
        if i == 6:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 6]) + ' and ' + str(variable) + ' < ' + str(array[i - 5]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df6 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5, df6
        if i == 7:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 7]) + ' and ' + str(variable) + ' < ' + str(array[i - 6]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 6]) + ' and ' + str(variable) + ' < ' + str(array[i - 5]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df6 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df7 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5, df6, df7
        if i == 8: 
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 8]) + ' and ' + str(variable) + ' < ' + str(array[i - 7]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 7]) + ' and ' + str(variable) + ' < ' + str(array[i - 6]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 6]) + ' and ' + str(variable) + ' < ' + str(array[i - 5]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df6 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df7 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df8 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5, df6, df7, df8
        if i == 9:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 9]) + ' and ' + str(variable) + ' < ' + str(array[i - 8]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 8]) + ' and ' + str(variable) + ' < ' + str(array[i - 7]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 7]) + ' and ' + str(variable) + ' < ' + str(array[i - 6]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 6]) + ' and ' + str(variable) + ' < ' + str(array[i - 5]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df6 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df7 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df8 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df9 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5, df6, df7, df8, df9
        if i == 10:
            df1 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 10])+ ' and ' + str(variable) + ' < ' + str(array[i - 9]) + '')
            df2 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 9]) + ' and ' + str(variable) + ' < ' + str(array[i - 8]) + '')
            df3 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 8]) + ' and ' + str(variable) + ' < ' + str(array[i - 7]) + '')
            df4 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 7]) + ' and ' + str(variable) + ' < ' + str(array[i - 6]) + '')
            df5 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 6]) + ' and ' + str(variable) + ' < ' + str(array[i - 5]) + '')
            df6 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 5]) + ' and ' + str(variable) + ' < ' + str(array[i - 4]) + '')
            df7 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 4]) + ' and ' + str(variable) + ' < ' + str(array[i - 3]) + '')
            df8 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 3]) + ' and ' + str(variable) + ' < ' + str(array[i - 2]) + '')
            df9 = inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 2]) + ' and ' + str(variable) + ' < ' + str(array[i - 1]) + '')
            df10= inputDataframe.query('' + str(variable) + ' > ' + str(array[i - 1]) + ' and ' + str(variable) + ' < ' + str(array[i])     + '')
            
            return df1, df2, df3, df4, df5, df6, df7, df8, df9, df10
        
        else:
            print('Need more or have too many entries in your array.')

In [ ]:
# Make the arrays needed to see the number of particles in a jet as a function of jet pT.

jets_pt_1 = np.array(['5','6','7','8','9','10','11','12','13','14','15'])
jets_pt_2 = np.array(['15','16','17','18','19','20','21','22','23','24','25'])
jets_pt_3 = np.array(['25','26','27','28','29','30'])

jet_mult_x     = np.array([5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5,
                           20.5, 21.5, 22.5, 23.5, 24.5, 25.5, 26.5, 27.5, 28.5, 29.5])
jet_mult_x_err = [0.5]*(jet_mult_x.shape[0])

In [ ]:
jets1_a ,jets2_a ,jets3_a ,jets4_a ,jets5_a ,jets6_a ,jets7_a ,jets8_a ,jets9_a ,jets10_a = bins(data,'pt',jets_pt_1,jets_pt_1.shape[0])
jets11_a,jets12_a,jets13_a,jets14_a,jets15_a,jets16_a,jets17_a,jets18_a,jets19_a,jets20_a = bins(data,'pt',jets_pt_2,jets_pt_2.shape[0])
jets21_a,jets22_a,jets23_a,jets24_a,jets25_a                                              = bins(data,'pt',jets_pt_3,jets_pt_3.shape[0])

In [ ]:
jets1_sd ,jets2_sd ,jets3_sd ,jets4_sd ,jets5_sd ,jets6_sd ,jets7_sd ,jets8_sd ,jets9_sd ,jets10_sd = bins(data,'pt',jets_pt_1,jets_pt_1.shape[0])
jets11_sd,jets12_sd,jets13_sd,jets14_sd,jets15_sd,jets16_sd,jets17_sd,jets18_sd,jets19_sd,jets20_sd = bins(data,'pt',jets_pt_2,jets_pt_2.shape[0])
jets21_sd,jets22_sd,jets23_sd,jets24_sd,jets25_sd                                                   = bins(data,'pt',jets_pt_3,jets_pt_3.shape[0])

In [ ]:
# Determine the mean of the number of particles in the jet per jet pT bin from the event generations.

jets1_mult_a  = np.mean(jets1_a['mult'])
jets2_mult_a  = np.mean(jets2_a['mult'])
jets3_mult_a  = np.mean(jets3_a['mult'])
jets4_mult_a  = np.mean(jets4_a['mult'])
jets5_mult_a  = np.mean(jets5_a['mult'])
jets6_mult_a  = np.mean(jets6_a['mult'])
jets7_mult_a  = np.mean(jets7_a['mult'])
jets8_mult_a  = np.mean(jets8_a['mult'])
jets9_mult_a  = np.mean(jets9_a['mult'])

jets10_mult_a = np.mean(jets10_a['mult'])
jets11_mult_a = np.mean(jets11_a['mult'])
jets12_mult_a = np.mean(jets12_a['mult'])
jets13_mult_a = np.mean(jets13_a['mult'])
jets14_mult_a = np.mean(jets14_a['mult'])
jets15_mult_a = np.mean(jets15_a['mult'])
jets16_mult_a = np.mean(jets16_a['mult'])
jets17_mult_a = np.mean(jets17_a['mult'])
jets18_mult_a = np.mean(jets18_a['mult'])
jets19_mult_a = np.mean(jets19_a['mult'])
jets20_mult_a = np.mean(jets20_a['mult'])
jets21_mult_a = np.mean(jets21_a['mult'])
jets22_mult_a = np.mean(jets22_a['mult'])
jets23_mult_a = np.mean(jets23_a['mult'])
jets24_mult_a = np.mean(jets24_a['mult'])
jets25_mult_a = np.mean(jets25_a['mult'])

jet_mult_a = np.array([jets1_mult_a,jets2_mult_a,jets3_mult_a,jets4_mult_a,jets5_mult_a,jets6_mult_a,jets7_mult_a,
                       jets8_mult_a,jets9_mult_a,jets10_mult_a,jets11_mult_a,jets12_mult_a,jets13_mult_a,
                       jets14_mult_a,jets15_mult_a,jets16_mult_a,jets17_mult_a,jets18_mult_a,jets19_mult_a,
                       jets20_mult_a,jets21_mult_a,jets22_mult_a,jets23_mult_a,jets24_mult_a,jets25_mult_a])

jet_mult_y_err_a = np.array([np.std(jets1_a['mult']),np.std(jets2_a['mult']),np.std(jets3_a['mult']),
                           np.std(jets4_a['mult']),np.std(jets5_a['mult']),np.std(jets6_a['mult']),
                           np.std(jets7_a['mult']),np.std(jets8_a['mult']),np.std(jets9_a['mult']),
                           np.std(jets10_a['mult']),np.std(jets11_a['mult']),np.std(jets12_a['mult']),
                           np.std(jets13_a['mult']),np.std(jets14_a['mult']),np.std(jets15_a['mult']),
                           np.std(jets16_a['mult']),np.std(jets17_a['mult']),np.std(jets18_a['mult']),
                           np.std(jets19_a['mult']),np.std(jets20_a['mult']),np.std(jets21_a['mult']),
                           np.std(jets22_a['mult']),np.std(jets23_a['mult']),np.std(jets24_a['mult']),
                           np.std(jets25_a['mult'])])

In [ ]:
# Determine the mean of the number of particles in the jet, after soft drop from the event generations, per pT bin.

jets1_mult_sd  = np.mean(jets1_sd['sd_mult'])
jets2_mult_sd  = np.mean(jets2_sd['sd_mult'])
jets3_mult_sd  = np.mean(jets3_sd['sd_mult'])
jets4_mult_sd  = np.mean(jets4_sd['sd_mult'])
jets5_mult_sd  = np.mean(jets5_sd['sd_mult'])
jets6_mult_sd  = np.mean(jets6_sd['sd_mult'])
jets7_mult_sd  = np.mean(jets7_sd['sd_mult'])
jets8_mult_sd  = np.mean(jets8_sd['sd_mult'])
jets9_mult_sd  = np.mean(jets9_sd['sd_mult'])

jets10_mult_sd = np.mean(jets10_sd['sd_mult'])
jets11_mult_sd = np.mean(jets11_sd['sd_mult'])
jets12_mult_sd = np.mean(jets12_sd['sd_mult'])
jets13_mult_sd = np.mean(jets13_sd['sd_mult'])
jets14_mult_sd = np.mean(jets14_sd['sd_mult'])
jets15_mult_sd = np.mean(jets15_sd['sd_mult'])
jets16_mult_sd = np.mean(jets16_sd['sd_mult'])
jets17_mult_sd = np.mean(jets17_sd['sd_mult'])
jets18_mult_sd = np.mean(jets18_sd['sd_mult'])
jets19_mult_sd = np.mean(jets19_sd['sd_mult'])
jets20_mult_sd = np.mean(jets20_sd['sd_mult'])
jets21_mult_sd = np.mean(jets21_sd['sd_mult'])
jets22_mult_sd = np.mean(jets22_sd['sd_mult'])
jets23_mult_sd = np.mean(jets23_sd['sd_mult'])
jets24_mult_sd = np.mean(jets24_sd['sd_mult'])
jets25_mult_sd = np.mean(jets25_sd['sd_mult'])

jet_mult_sd = np.array([jets1_mult_sd,jets2_mult_sd,jets3_mult_sd,jets4_mult_sd,jets5_mult_sd,jets6_mult_sd,jets7_mult_sd,
                       jets8_mult_sd,jets9_mult_sd,jets10_mult_sd,jets11_mult_sd,jets12_mult_sd,jets13_mult_sd,
                       jets14_mult_sd,jets15_mult_sd,jets16_mult_sd,jets17_mult_sd,jets18_mult_sd,jets19_mult_sd,
                       jets20_mult_sd,jets21_mult_sd,jets22_mult_sd,jets23_mult_sd,jets24_mult_sd,jets25_mult_sd])

jet_mult_y_err_sd = np.array([np.std(jets1_sd['sd_mult']),np.std(jets2_sd['sd_mult']),np.std(jets3_sd['sd_mult']),
                           np.std(jets4_sd['sd_mult']),np.std(jets5_sd['sd_mult']),np.std(jets6_sd['sd_mult']),
                           np.std(jets7_sd['sd_mult']),np.std(jets8_sd['sd_mult']),np.std(jets9_sd['sd_mult']),
                           np.std(jets10_sd['sd_mult']),np.std(jets11_sd['sd_mult']),np.std(jets12_sd['sd_mult']),
                           np.std(jets13_sd['sd_mult']),np.std(jets14_sd['sd_mult']),np.std(jets15_sd['sd_mult']),
                           np.std(jets16_sd['sd_mult']),np.std(jets17_sd['sd_mult']),np.std(jets18_sd['sd_mult']),
                           np.std(jets19_sd['sd_mult']),np.std(jets20_sd['sd_mult']),np.std(jets21_sd['sd_mult']),
                           np.std(jets22_sd['sd_mult']),np.std(jets23_sd['sd_mult']),np.std(jets24_sd['sd_mult']),
                           np.std(jets25_sd['sd_mult'])])

In [ ]:
# Look at the differences before and after soft drop.

fig = plt.figure(figsize=(8,7))

plt.errorbar(jet_mult_x, jet_mult_a, xerr=jet_mult_x_err, yerr=jet_mult_y_err_a, fmt='o', alpha = 0.7, color = 'red', label='Before Soft-Drop')
plt.errorbar(jet_mult_x, jet_mult_sd, xerr=jet_mult_x_err, yerr=jet_mult_y_err_sd, fmt='o', alpha = 0.7, color = 'blue', label = 'After Soft-Drop')

plt.plot([],[],'',label=r'$\mathrm{DIS \: configuration \: e-p}$', color='white')
plt.plot([],[],'',label=r'$\mathrm{E}_{beam}^{e^{-}} \: = ' + elabel + ' \: \mathrm{GeV}, \: \mathrm{E}_{beam}^{p^{+}} \: = ' + plabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.xlabel(r'$\mathrm{p}_\mathrm{T}^{\mathrm{jet}} \:\: (\mathrm{GeV}/c)$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(0,25)
plt.ylabel(r'$\mathrm{Number \:\: of \:\: Constituents}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.ylim(0,25)
plt.legend(loc='upper left', frameon=False, prop={'size':15})
# plt.savefig('ep_sim_jet_N_pT.pdf')

In [ ]:
# Now we split the distributions into different eta bins that will allows to see what technologies should be 
# used to analyze jets with properties of interest.

print 'STD of x for  -1.3 < eta < -1.0:',np.std(data.query('-1.3 < eta < -1.0')['x'])
print 'Mean of x for -1.3 < eta < -1.0:',np.mean(data.query('-1.3 < eta < -1.0')['x'])
print '\n'
print 'STD of x for  -1.6 < eta < -1.3:',np.std(data.query('-1.6 < eta < -1.3')['x'])
print 'Mean of x for -1.6 < eta < -1.3:',np.mean(data.query('-1.6 < eta < -1.3')['x'])
print '\n'
print 'STD of x for  -2.0 < eta < -1.6:',np.std(data.query('-2.0 < eta < -1.6')['x'])
print 'Mean of x for -2.0 < eta < -1.6:',np.mean(data.query('-2.0 < eta < -1.6')['x'])

|     bins  |  SD of x  |  Mean of x |
|-----------|-------|-------|
|  -1.3 < eta < -1.0  | 0.005| 0.004|
|-1.6 <  eta < -1.3  | 0.005| 0.003|
|-2.0 <  eta < -1.6  | 0.006| 0.004|

In [ ]:
fig, ax = plt.subplots(figsize=(9,7))

fig.suptitle(r'$\mathrm{E}_{beam}^{e^{-}} \: = ' + elabel + ' \: \mathrm{GeV}, \: \mathrm{E}_{beam}^{p^{+}} \: = ' + plabel + ', \: \mathrm{and} \: Q^2_{min} \: = \: 1 \: \mathrm{GeV}^2$',fontsize = 20)
# plotData(data.query('eta < -1.0')['e'], bins = 40, arange=(0,30), label = r'$ -1.0 > \eta$', color = 'purple', normed = True)
plotData(data.query('-1.3 < eta < -1.0')['e'], bins = 40, arange=(0,30), label = r'$-1.3 \: < \: \eta \: > \: -1.0$', color = 'red', normed = True)
plotData(data.query('-1.6 < eta < -1.3')['e'], bins = 40, arange=(0,30), label = r'$-1.6 \: < \: \eta \: < \: -1.3$', color = 'orange', normed = True)
plotData(data.query('-2.0 < eta < -1.6')['e'], bins = 40, arange=(0,30), label = r'$-2.0 \: < \: \eta \: < \: -1.6$', color = 'blue', fillstyle = 'none',normed = True)
# plotData(data.query('-1.0 < eta < 0.0')['e'], bins = 20, arange=(0,100), label = r'$-1.0 \:< \:\eta \: < \: 0.0$', color = 'green', normed = True)
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \: > \: 5 \:\mathrm{GeV}/c$', color='white')

plt.xlabel(r'$\mathrm{E}_\mathrm{jet} \: (\mathrm{GeV})$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(5,25)
plt.ylabel(r'$\mathrm{d} \sigma  \: \times \: \mathrm{10} \: \mathrm{fb^{-1}}$', fontsize = 25)
plt.yscale("log")
plt.yticks(fontsize =17)
plt.minorticks_on()

handles, labels = ax.get_legend_handles_labels()

# sort both labels and handles by labels
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles, labels, bbox_to_anchor=(1, 1),frameon=False, prop={'size':15})
# plt.savefig('ep_sim_jet_E_etaBins.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(9,7))

# plotData(data.query('eta < -1.0')['pt'], bins = 60,arange=(0,20), label = r'$ -1.0 \: > \: \eta$', color = 'purple', normed = True)
plotData(data.query('-1.3 < eta < -1.0')['pt'], bins = 60, fillstyle='none', arange=(0,20), label = r'$ -1.3 \: < \: \eta \: < \: -1.0$', color = 'red', normed = True)
plotData(data.query('-1.6 < eta < -1.3')['pt'], bins = 60, arange=(0,20), label = r'$ -1.6 \: < \: \eta \: < \: -1.3$', color = 'orange', normed = True)
plotData(data.query('-2.0 < eta < -1.6')['pt'], bins = 60, arange=(0,20), label = r'$ -2.0 \: < \: \eta \: < \: -1.6$', fillstyle = 'none', color = 'blue', normed = True)
# plotData(data.query('2.0 < eta')['pt'], bins = 20, arange=(0,50), label = r'$\eta \: > \: 2.0$', color = 'red',fillstyle = 'none', normed = True)
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \: > \: 5 \:\mathrm{GeV}/c$', color='white')

plt.xlabel(r'$\mathrm{p}_\mathrm{T}^{\mathrm{jet}} \:\: (\mathrm{GeV}/c)$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(5,15)
plt.ylabel(r'$\mathrm{d} \sigma  \: \times \: \mathrm{10} \: \mathrm{fb^{-1}}$', fontsize = 25)
plt.yscale("log")
plt.yticks(fontsize =17)
plt.minorticks_on()

handles, labels = ax.get_legend_handles_labels()

# Sort both labels and handles by labels
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles, labels, frameon=False, prop={'size':15})
# plt.savefig('ep_sim_jet_pT_etaBins.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(9,7))

plotData(data.query('1 < Q2 < 3')['pt'], bins = 30, fillstyle='none', arange=(5,20), label = r'$ 1.0 \: < \: Q^{2} \: < \: 3.0 \: \mathrm{GeV}^{2}$', color = 'red', normed = True)
plotData(data.query('3 < Q2 < 10')['pt'], bins = 30, arange=(5,20), label = r'$ 3.0 \: < \: Q^{2} \: < \: 10.0 \: \mathrm{GeV}^{2}$', color = 'orange', normed = True)
plotData(data.query('10 < Q2 ')['pt'], bins = 30, arange=(5,20), label = r'$ 10.0 \: \mathrm{GeV}^{2} \: < \: Q^{2} $', fillstyle = 'none', color = 'blue', normed = True)
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \: > \: 5 \:\mathrm{GeV}/c$', color='white')

plt.xlabel(r'$\mathrm{p}_\mathrm{T}^{\mathrm{jet}} \:\: (\mathrm{GeV}/c)$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(5,20)
plt.ylabel(r'$\mathrm{d} \sigma  \: \times \: \mathrm{10} \: \mathrm{fb^{-1}}$', fontsize = 25)
plt.yscale("log")
plt.yticks(fontsize =17)
plt.minorticks_on()

handles, labels = ax.get_legend_handles_labels()

# Sort both labels and handles by labels
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles, labels, frameon=False, prop={'size':15})
# plt.savefig('ep_sim_jet_pT_etaBins.pdf')

In [ ]:
# Ratio to electron pT is centered near 1 since the come back-to-back. 

data.eval('pTratio = pt / lpt', inplace = True)
fig, ax = plt.subplots(figsize=(9,7))

plotData(data.query('lpt > 10')['pTratio'], bins = 31, arange=(0,2), color = 'purple', normed = True)
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{jet} \: > \: 5 \:\mathrm{GeV}/c$', color='white')
plt.plot([],[],'',label=r'$\mathrm{p}_\mathrm{T}^\mathrm{scat. e} \: > \: 10 \:\mathrm{GeV}/c$', color='white')

plt.xlabel(r'$\mathrm{p}_\mathrm{T}^{\mathrm{jet}} \: / \: \mathrm{p}_\mathrm{T}^{\mathrm{scat. e}}$', fontsize = 25)
plt.xticks(fontsize = 17)
plt.xlim(0,2)
plt.ylabel(r'$\mathrm{d} \sigma  \: \times \: \mathrm{10} \: \mathrm{fb^{-1}}$', fontsize = 25)
# plt.yscale("log")
plt.yticks(fontsize =17)
plt.minorticks_on()
plt.legend(loc="upper right",frameon= False, prop={'size': 17})
plt.axvline(x=1.)
ax.yaxis.set_major_formatter(formatter)

In [ ]:
# Similar study as before for the number of particles in the jets. But, now a complementary picture is shown as
# a function of e for different x bins.

data_x1 = data.query('x < 0.01')
data_x2 = data.query('x > 0.1')

jets_pt_1 = np.array(['5','10','15','20','25','30','35','40','45','50','55'])
jets_pt_2 = np.array(['55','60','65','70','75','80','85','90','95','100','105'])
jets_pt_3 = np.array(['105','110','115','120','125','130'])

jet_mult_x     = np.array([7.5, 12.5, 17.5, 22.5, 27.5, 32.5, 37.5, 42.5, 47.5, 52.5, 57.5, 62.5, 67.5, 72.5, 77.5,
                           82.5, 87.5, 92.5, 97.5, 102.5, 107.5, 112.5, 117.5, 122.5, 127.5])
jet_mult_x_err = [2.5]*(jet_mult_x.shape[0])

jets1_a ,jets2_a ,jets3_a ,jets4_a ,jets5_a ,jets6_a ,jets7_a ,jets8_a ,jets9_a ,jets10_a = bins(data_x1,'e',jets_pt_1,jets_pt_1.shape[0])
jets11_a,jets12_a,jets13_a,jets14_a,jets15_a,jets16_a,jets17_a,jets18_a,jets19_a,jets20_a = bins(data_x1,'e',jets_pt_2,jets_pt_2.shape[0])
jets21_a,jets22_a,jets23_a,jets24_a,jets25_a                                              = bins(data_x1,'e',jets_pt_3,jets_pt_3.shape[0])

jets1_c ,jets2_c ,jets3_c ,jets4_c ,jets5_c ,jets6_c ,jets7_c ,jets8_c ,jets9_c, jets10_c = bins(data_x2,'e',jets_pt_1,jets_pt_1.shape[0])
jets11_c,jets12_c,jets13_c,jets14_c,jets15_c,jets16_c,jets17_c,jets18_c,jets19_c,jets20_c = bins(data_x2,'e',jets_pt_2,jets_pt_2.shape[0])
jets21_c,jets22_c,jets23_c,jets24_c,jets25_c                                              = bins(data_x2,'e',jets_pt_3,jets_pt_3.shape[0])

In [ ]:
jets1_mult_a  = np.mean(jets1_a['mult'])
jets2_mult_a  = np.mean(jets2_a['mult'])
jets3_mult_a  = np.mean(jets3_a['mult'])
jets4_mult_a  = np.mean(jets4_a['mult'])
jets5_mult_a  = np.mean(jets5_a['mult'])
jets6_mult_a  = np.mean(jets6_a['mult'])
jets7_mult_a  = np.mean(jets7_a['mult'])
jets8_mult_a  = np.mean(jets8_a['mult'])
jets9_mult_a  = np.mean(jets9_a['mult'])

jets10_mult_a = np.mean(jets10_a['mult'])
jets11_mult_a = np.mean(jets11_a['mult'])
jets12_mult_a = np.mean(jets12_a['mult'])
jets13_mult_a = np.mean(jets13_a['mult'])
jets14_mult_a = np.mean(jets14_a['mult'])
jets15_mult_a = np.mean(jets15_a['mult'])
jets16_mult_a = np.mean(jets16_a['mult'])
jets17_mult_a = np.mean(jets17_a['mult'])
jets18_mult_a = np.mean(jets18_a['mult'])
jets19_mult_a = np.mean(jets19_a['mult'])
jets20_mult_a = np.mean(jets20_a['mult'])
jets21_mult_a = np.mean(jets21_a['mult'])
jets22_mult_a = np.mean(jets22_a['mult'])
jets23_mult_a = np.mean(jets23_a['mult'])
jets24_mult_a = np.mean(jets24_a['mult'])
jets25_mult_a = np.mean(jets25_a['mult'])

jet_mult_a = np.array([jets1_mult_a,jets2_mult_a,jets3_mult_a,jets4_mult_a,jets5_mult_a,jets6_mult_a,jets7_mult_a,
                       jets8_mult_a,jets9_mult_a,jets10_mult_a,jets11_mult_a,jets12_mult_a,jets13_mult_a,
                       jets14_mult_a,jets15_mult_a,jets16_mult_a,jets17_mult_a,jets18_mult_a,jets19_mult_a,
                       jets20_mult_a,jets21_mult_a,jets22_mult_a,jets23_mult_a,jets24_mult_a,jets25_mult_a])

jet_mult_y_err_a = np.array([np.std(jets1_a['mult']),np.std(jets2_a['mult']),np.std(jets3_a['mult']),
                           np.std(jets4_a['mult']),np.std(jets5_a['mult']),np.std(jets6_a['mult']),
                           np.std(jets7_a['mult']),np.std(jets8_a['mult']),np.std(jets9_a['mult']),
                           np.std(jets10_a['mult']),np.std(jets11_a['mult']),np.std(jets12_a['mult']),
                           np.std(jets13_a['mult']),np.std(jets14_a['mult']),np.std(jets15_a['mult']),
                           np.std(jets16_a['mult']),np.std(jets17_a['mult']),np.std(jets18_a['mult']),
                           np.std(jets19_a['mult']),np.std(jets20_a['mult']),np.std(jets21_a['mult']),
                           np.std(jets22_a['mult']),np.std(jets23_a['mult']),np.std(jets24_a['mult']),
                           np.std(jets25_a['mult'])])

In [ ]:
jets1_mult_c  = np.mean(jets1_c['mult'])
jets2_mult_c  = np.mean(jets2_c['mult'])
jets3_mult_c  = np.mean(jets3_c['mult'])
jets4_mult_c  = np.mean(jets4_c['mult'])
jets5_mult_c  = np.mean(jets5_c['mult'])
jets6_mult_c  = np.mean(jets6_c['mult'])
jets7_mult_c  = np.mean(jets7_c['mult'])
jets8_mult_c  = np.mean(jets8_c['mult'])
jets9_mult_c  = np.mean(jets9_c['mult'])

jets10_mult_c = np.mean(jets10_c['mult'])
jets11_mult_c = np.mean(jets11_c['mult'])
jets12_mult_c = np.mean(jets12_c['mult'])
jets13_mult_c = np.mean(jets13_c['mult'])
jets14_mult_c = np.mean(jets14_c['mult'])
jets15_mult_c = np.mean(jets15_c['mult'])
jets16_mult_c = np.mean(jets16_c['mult'])
jets17_mult_c = np.mean(jets17_c['mult'])
jets18_mult_c = np.mean(jets18_c['mult'])
jets19_mult_c = np.mean(jets19_c['mult'])
jets20_mult_c = np.mean(jets20_c['mult'])
jets21_mult_c = np.mean(jets21_c['mult'])
jets22_mult_c = np.mean(jets22_c['mult'])
jets23_mult_c = np.mean(jets23_c['mult'])
jets24_mult_c = np.mean(jets24_c['mult'])
jets25_mult_c = np.mean(jets25_c['mult'])

jet_mult_c = np.array([jets1_mult_c,jets2_mult_c,jets3_mult_c,jets4_mult_c,jets5_mult_c,jets6_mult_c,jets7_mult_c,
                       jets8_mult_c,jets9_mult_c,jets10_mult_c,jets11_mult_c,jets12_mult_c,jets13_mult_c,
                       jets14_mult_c,jets15_mult_c,jets16_mult_c,jets17_mult_c,jets18_mult_c,jets19_mult_c,
                       jets20_mult_c,jets21_mult_c,jets22_mult_c,jets23_mult_c,jets24_mult_c,jets25_mult_c])

jet_mult_y_err_c = np.array([np.std(jets1_c['mult']),np.std(jets2_c['mult']),np.std(jets3_c['mult']),
                           np.std(jets4_c['mult']) ,np.std(jets5_c['mult']) ,np.std(jets6_c['mult']),
                           np.std(jets7_c['mult']) ,np.std(jets8_c['mult']) ,np.std(jets9_c['mult']),
                           np.std(jets10_c['mult']),np.std(jets11_c['mult']),np.std(jets12_c['mult']),
                           np.std(jets13_c['mult']),np.std(jets14_c['mult']),np.std(jets15_c['mult']),
                           np.std(jets16_c['mult']),np.std(jets17_c['mult']),np.std(jets18_c['mult']),
                           np.std(jets19_c['mult']),np.std(jets20_c['mult']),np.std(jets21_c['mult']),
                           np.std(jets22_c['mult']),np.std(jets23_c['mult']),np.std(jets24_c['mult']),
                           np.std(jets25_c['mult'])])

In [ ]:
fig = plt.figure(figsize=(8,7))

plt.errorbar(jet_mult_x, jet_mult_a, xerr=jet_mult_x_err, yerr=jet_mult_y_err_a, fmt='o', color = 'blue',label=r'$x \: < \: 0.01$')
plt.errorbar(jet_mult_x, jet_mult_c, xerr=jet_mult_x_err, yerr=jet_mult_y_err_c, fmt='o', color = 'red', label=r'$x \: > \: 0.1$')
plt.plot([],[],'',label=r'$\mathrm{DIS \: configuration \: e-p}$', color='white')
plt.plot([],[],'',label=r'$\mathrm{E}_{beam}^{e^{-}} \: = ' + elabel + ' \: \mathrm{GeV}, \: \mathrm{E}_{beam}^{p^{+}} \: = ' + plabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label=r'$\sqrt{s}\:=\:' + slabel + ' \: \mathrm{GeV}$', color='white')
plt.plot([],[],'',label='' + algolabel + ', R=1.0', color='white')
plt.xlabel(r'$\mathrm{E}_{\mathrm{jet}} \:\: (\mathrm{GeV})$', fontsize = 25)
plt.xticks(fontsize = 17)
# plt.xlim(0,30)
plt.ylabel(r'$\mathrm{Number \:\: of \:\: Constituents}$', fontsize = 20)
plt.yticks(fontsize =17)
plt.ylim(0,25)
plt.legend(loc='upper left', frameon=False, prop={'size':15})